In [8]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore")

In [9]:
df_directorio = pd.read_excel("DIRECTORIO DE OFICINAS.xlsx")
df_base = pd.read_excel("BASE CORREOS.xlsx")

In [10]:
df_directorio.head(3)
print('df_directorio: ', df_directorio.shape)

,ID Oficina,Oficina,Registro,Nombre,Correo,Puesto,División
0,648,OFICINA CHOTA,P020779,"ESPINOZA MONSALVE, EDWIN OMAR",EDWIN.ESPINOZA@BBVA.COM,(B217)GERENTE DE OFICINA F,DIVISION CENTRO ORIENTE
1,654,OFICINA PLAZA DEL SOL HUACHO,P009933,"TAPIA CERNA, SHEILLA AMELIA",STAPIA@BBVA.COM,(B217)GERENTE DE OFICINA F,DIVISION LIMA NORTE
2,659,OFICINA SEBASTIÁN BARRANCA,P017610,"GUINET CABRAL, JOHN JANCEL",JGUINET@BBVA.COM,(B211)GERENTE DE OFICINA A3,DIVISION LIMA CENTRO


In [4]:
df_base.head(3)
print('df_base: ', df_base.shape)

,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com, eyttus...",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,5,"GCASTROP@BBVA.COM, buzon-procesal-sin-garantia...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P011496,"CASTRO PAZ, GALBA"
2,10,"evivas@bbva.com, richard.ordaya@bbva.com, paul...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P013079,"VILLEGAS HUAMANI, NELLA"


In [12]:
df_directorio.isna().sum()

ID Oficina    0
Oficina       0
Registro      0
Nombre        0
Correo        0
Puesto        0
División      0
dtype: int64

In [13]:
df_base.isna().sum()

OFICINA           0
CORREOS           0
Territorio        0
Nombre Oficina    0
Cargo             0
Registro          0
Nombre            0
dtype: int64

In [14]:
df_directorio.dropna(inplace=True)
df_base.dropna(inplace=True)

print('df_directorio: ', df_directorio.shape)
print('df_base: ', df_base.shape)

df_directorio:  (330, 7)
df_base:  (332, 7)


In [15]:
columnas_directorio = ['ID Oficina', 'Puesto', 'Correo', 'Registro', 'Nombre']
df_directorio = df_directorio[columnas_directorio]
df_directorio.rename(columns={'ID Oficina': 'OFICINA'}, inplace=True)
df_directorio.head(3)

,OFICINA,Puesto,Correo,Registro,Nombre
0,648,(B217)GERENTE DE OFICINA F,EDWIN.ESPINOZA@BBVA.COM,P020779,"ESPINOZA MONSALVE, EDWIN OMAR"
1,654,(B217)GERENTE DE OFICINA F,STAPIA@BBVA.COM,P009933,"TAPIA CERNA, SHEILLA AMELIA"
2,659,(B211)GERENTE DE OFICINA A3,JGUINET@BBVA.COM,P017610,"GUINET CABRAL, JOHN JANCEL"


In [16]:
df_directorio = df_directorio[df_directorio['Puesto'].str.contains('GERENTE', case=False)].reset_index(drop=True)
df_directorio.head(3)

,OFICINA,Puesto,Correo,Registro,Nombre
0,648,(B217)GERENTE DE OFICINA F,EDWIN.ESPINOZA@BBVA.COM,P020779,"ESPINOZA MONSALVE, EDWIN OMAR"
1,654,(B217)GERENTE DE OFICINA F,STAPIA@BBVA.COM,P009933,"TAPIA CERNA, SHEILLA AMELIA"
2,659,(B211)GERENTE DE OFICINA A3,JGUINET@BBVA.COM,P017610,"GUINET CABRAL, JOHN JANCEL"


In [17]:
df_directorio['Puesto'] = df_directorio['Puesto'].replace({
    '(?i).*sub gerente.*': 2,
    '(?i).*gerente.*': 1
}, regex=True)
df_directorio.sort_values(by=['OFICINA','Puesto'], inplace=True, ignore_index=True)
df_directorio.reset_index(drop=True, inplace=True)
df_directorio.head(10)

,OFICINA,Puesto,Correo,Registro,Nombre
0,5,2,KGALVEZ@BBVA.COM,P008733,"GALVEZ AGUIRRE, CATHERINE JULIANA"
1,10,2,RICHARD.ORDAYA@BBVA.COM,P024186,"ORDAYA QUISPE, RICHARD JHONY"
2,13,2,JJOFERNANDEZ@BBVA.COM,P012584,"FERNANDEZ SANDOVAL, JAIR JOAO"
3,14,2,LAQUINO@BBVA.COM,P013132,"AQUINO LOPEZ, LIZETH NANCY"
4,23,2,NESTOR.ROJAS@BBVA.COM,P020086,"ROJAS BURGA, NESTOR EDUARDO"
5,30,2,MACALDERON@BBVA.COM,P014467,"CALDERON TORRES, MELANIA ANHELI"
6,38,2,HUGO.QUILCA@BBVA.COM,P032969,"QUILCA VERA, HUGO GABRIEL"
7,48,2,KATERINE.RODRIGUEZ@BBVA.COM,P027880,"RODRIGUEZ AREDO, KATERINE ALEXANDRA"
8,50,2,IROJASO@BBVA.COM,P017201,"ROJAS OCHOA, IRIS DEL PILAR"
9,59,2,KATIA.CASTILLO@BBVA.COM,P024560,"CASTILLO REQUEJO, KATIA CECILIA"


In [18]:
def concatenar_correos(grupo: pd.DataFrame) -> pd.Series:
    gerente_correo = grupo.loc[grupo['Puesto']==1, 'Correo'].values
    subgerente_correo = grupo.loc[grupo['Puesto']==2, 'Correo'].values
    
    if len(gerente_correo) > 0 and len(subgerente_correo) > 0:
        correos_concatenados = gerente_correo[0] + ', ' + subgerente_correo[0]
    elif len(gerente_correo) > 0:
        correos_concatenados = gerente_correo[0]
    else:
        correos_concatenados = subgerente_correo[0]
    
    return pd.Series({
        'OFICINA': grupo['OFICINA'].iloc[0],
        'Puesto': 'GERENTE DE OFICINA',
        'Correo': correos_concatenados,
        'Registro': grupo['Registro'].iloc[0],
        'Nombre': grupo['Nombre'].iloc[0]
        })

df_final = df_directorio.groupby('OFICINA').apply(concatenar_correos).reset_index(drop=True)

print(df_final.shape)
df_final.head(3)

(288, 5)


,OFICINA,Puesto,Correo,Registro,Nombre
0,5,GERENTE DE OFICINA,KGALVEZ@BBVA.COM,P008733,"GALVEZ AGUIRRE, CATHERINE JULIANA"
1,10,GERENTE DE OFICINA,RICHARD.ORDAYA@BBVA.COM,P024186,"ORDAYA QUISPE, RICHARD JHONY"
2,13,GERENTE DE OFICINA,JJOFERNANDEZ@BBVA.COM,P012584,"FERNANDEZ SANDOVAL, JAIR JOAO"


In [19]:
df_actualizado = pd.merge(df_base, df_final, on='OFICINA', how='left')
print(df_actualizado.shape)
df_actualizado.head(3)

(332, 11)


,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro_x,Nombre_x,Puesto,Correo,Registro_y,Nombre_y
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com, mirko....",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA",NaN,NaN,NaN,NaN
1,5,"jvegac@bbva.com, kgalvez@bbva.com, mirko.iriar...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P011496,"CASTRO PAZ, GALBA",GERENTE DE OFICINA,KGALVEZ@BBVA.COM,P008733,"GALVEZ AGUIRRE, CATHERINE JULIANA"
2,10,"evivas@bbva.com, richard.ordaya@bbva.com, mirk...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P013079,"VILLEGAS HUAMANI, NELLA",GERENTE DE OFICINA,RICHARD.ORDAYA@BBVA.COM,P024186,"ORDAYA QUISPE, RICHARD JHONY"


In [20]:
df_actualizado.isna().sum()

OFICINA            0
CORREOS            0
Territorio         0
Nombre Oficina     0
Cargo              0
Registro_x         0
Nombre_x           0
Puesto            55
Correo            55
Registro_y        55
Nombre_y          55
dtype: int64

In [21]:
df_actualizado['Registro'] = df_actualizado['Registro_y'].fillna(df_actualizado['Registro_x'])
df_actualizado['Nombre'] = df_actualizado['Nombre_y'].fillna(df_actualizado['Nombre_x'])
df_actualizado.drop(columns=['Registro_x', 'Registro_y', 'Nombre_x', 'Nombre_y'], inplace=True)

In [22]:
df_actualizado['CORREOS'] = df_actualizado['Correo'].fillna(df_actualizado['CORREOS'])
df_actualizado.drop(columns=['Puesto', 'Correo'], inplace=True)
df_actualizado['CORREOS'] = df_actualizado['CORREOS'].str.lower()
print(df_actualizado.shape)
df_actualizado.head(3)

(332, 7)


,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,2,"teyzaguirre@bbva.com, dmalaga@bbva.com, mirko....",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,5,kgalvez@bbva.com,GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P008733,"GALVEZ AGUIRRE, CATHERINE JULIANA"
2,10,richard.ordaya@bbva.com,GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P024186,"ORDAYA QUISPE, RICHARD JHONY"


In [23]:
df_actualizado['OFICINA'] = df_actualizado['OFICINA'].astype(str).str.zfill(4)
df_actualizado.head(3)

,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,0002,"teyzaguirre@bbva.com, dmalaga@bbva.com, mirko....",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,0005,kgalvez@bbva.com,GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P008733,"GALVEZ AGUIRRE, CATHERINE JULIANA"
2,0010,richard.ordaya@bbva.com,GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P024186,"ORDAYA QUISPE, RICHARD JHONY"


In [25]:
df_actualizado.isna().sum()

OFICINA           0
CORREOS           0
Territorio        0
Nombre Oficina    0
Cargo             0
Registro          0
Nombre            0
dtype: int64

In [26]:
correos_nuevos = ['mirko.iriarte@bbva.com', 'paula.zuniga@bbva.com']

def agregar_correos(lista_correos: list) -> str:
    for correo in correos_nuevos:
        if correo not in lista_correos:
            lista_correos.append(correo)
    return ', '.join(lista_correos)

df_actualizado['CORREOS'] = df_actualizado['CORREOS'].str.split(', ').apply(agregar_correos)
df_actualizado.head(3)

,OFICINA,CORREOS,Territorio,Nombre Oficina,Cargo,Registro,Nombre
0,0002,"teyzaguirre@bbva.com, dmalaga@bbva.com, mirko....",GT CALLAO SAN MIGUEL,OFICINA PANDO,GERENTE DE OFICINA,P015142,"EYZAGUIRRE GONZALES, TATIANA"
1,0005,"kgalvez@bbva.com, mirko.iriarte@bbva.com, paul...",GT NORTE CHICO,OFICINA MEGAPLAZA CHIMBOTE,GERENTE DE OFICINA,P008733,"GALVEZ AGUIRRE, CATHERINE JULIANA"
2,0010,"richard.ordaya@bbva.com, mirko.iriarte@bbva.co...",GT CENTRO,OFICINA ANDAHUAYLAS,GERENTE DE OFICINA,P024186,"ORDAYA QUISPE, RICHARD JHONY"


In [27]:
df_actualizado.to_excel('CORREOS.xlsx', index=False)
os.startfile('CORREOS.xlsx')